# Build a QA system over SQL data

In [16]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Customer LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'Luís', 'Gonçalves', 'Embraer - Empresa Brasileira de Aeronáutica S.A.', 'Av. Brigadeiro Faria Lima, 2170', 'São José dos Campos', 'SP', 'Brazil', '12227-000', '+55 (12) 3923-5555', '+55 (12) 3923-5566', 'luisg@embraer.com.br', 3), (2, 'Leonie', 'Köhler', None, 'Theodor-Heuss-Straße 34', 'Stuttgart', None, 'Germany', '70174', '+49 0711 2842222', None, 'leonekohler@surfeu.de', 5), (3, 'François', 'Tremblay', None, '1498 rue Bélanger', 'Montréal', 'QC', 'Canada', 'H2G 1A7', '+1 (514) 721-4711', None, 'ftremblay@gmail.com', 3), (4, 'Bjørn', 'Hansen', None, 'Ullevålsveien 14', 'Oslo', None, 'Norway', '0171', '+47 22 44 22 22', None, 'bjorn.hansen@yahoo.no', 4), (5, 'František', 'Wichterlová', 'JetBrains s.r.o.', 'Klanova 9/506', 'Prague', None, 'Czech Republic', '14700', '+420 2 4172 5555', '+420 2 4172 5555', 'frantisekw@jetbrains.com', 4), (6, 'Helena', 'Holý', None, 'Rilská 3174/6', 'Prague', None, 'Czech Republic', '14300', '+420 2 4177 0449', None, 'hholy@gmail.com', 5), (7, 'As

## Chains

### Convert question to SQL query

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1")

In [13]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many customers are there?"})
response

'SELECT COUNT(CustomerId) FROM Customer'

`create_sql_query_chain` didn't the find Employee table, so failed to write SQL. So we use Customer instead.

In [14]:
response = 'SELECT COUNT(CustomerId) FROM Customer'

In [15]:
db.run(response)

'[(59,)]'

In [10]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

### Execute SQL query

Dangerous: Consider adding a human approval

In [18]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | execute_query
chain.invoke({"question": "How many customers are there"})

'[(59,)]'

### Answer the question

In [19]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

chain.invoke({"question": "How many customers are there?"})

'There are 59 customers.'

## Agents

In [22]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10be915e0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10be915e0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x10be915e0>),
 QuerySQLCheckerTool(description='Use this tool to double check if your 

### System Prompt

In [23]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

### Initializing agent

In [24]:
%%capture --no-stderr
%pip install --upgrade --quiet langgraph

In [26]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools, state_modifier=system_message)

In [29]:
for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Describe the playlisttrack table")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-11-20T05:40:58.106798Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'sql_db_list_tables', 'arguments': {}}}, {'function': {'name': 'sql_db_schema', 'arguments': {'table_names': 'playlisttrack'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 11597157625, 'load_duration': 35451667, 'prompt_eval_count': 716, 'prompt_eval_duration': 5179000000, 'eval_count': 106, 'eval_duration': 6368000000}, id='run-76cd6eaa-309c-41e1-94d0-d08aaa4e074a-0', tool_calls=[{'name': 'sql_db_list_tables', 'args': {}, 'id': 'fcc139f6-c14e-40d5-b93d-8fc1bb025d61', 'type': 'tool_call'}, {'name': 'sql_db_schema', 'args': {'table_names': 'playlisttrack'}, 'id': 'e12a7e78-86de-44ee-b797-037fc3dc545f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 716, 'output_tokens': 106, 'total_tokens': 822})]}}
----
{'tools': {'messages': [Too

### Dealing with high-cardinality columns

In [30]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))


artists = query_as_list(db, "SELECT Name FROM Artist")
albums = query_as_list(db, "SELECT Title FROM Album")
albums[:5]

['B-Sides -',
 'Achtung Baby',
 'Audioslave',
 'Quanta Gente Veio ver--Bônus De Carnaval',
 'Berlioz: Symphonie Fantastique']

The agent can execute a retriever tool at this discretion.

In [31]:
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

model = "nomic-embed-text:latest"
vector_db = FAISS.from_texts(artists + albums, OllamaEmbeddings(model=model))
retriever = vector_db.as_retriever(search_kwargs={"k": 5})
description = """Use to look up values to filter on. Input is an approximate spelling of the proper noun, output is \
valid proper nouns. Use the noun most similar to the search."""

retriever_tool = create_retriever_tool(
    retriever,
    name="search_proper_nouns",
    description=description
)

In [32]:
print(retriever_tool.invoke("Alice Chains"))

Alice In Chains

Corinne Bailey Rae

Queen

AC/DC

Guns N' Roses


In [34]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

You have access to the following tables: {table_names}

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool!
Do not try to guess at the proper name - use this function to find similar ones.""".format(
    table_names=db.get_usable_table_names()
)

system_message = SystemMessage(content=system)

tools.append(retriever_tool)

agent = create_react_agent(llm, tools, state_modifier=system_message)

In [35]:
for s in agent.stream(
    {"messages": [HumanMessage(content="How many albums does alis in chain have?")]}
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2024-11-20T05:53:29.619789Z', 'message': {'role': 'assistant', 'content': '', 'tool_calls': [{'function': {'name': 'search_proper_nouns', 'arguments': {'query': 'alis in chain'}}}]}, 'done_reason': 'stop', 'done': True, 'total_duration': 14219401791, 'load_duration': 583231750, 'prompt_eval_count': 855, 'prompt_eval_duration': 10463000000, 'eval_count': 48, 'eval_duration': 2902000000}, id='run-dd80594b-e045-47fe-8f0d-993024b490f7-0', tool_calls=[{'name': 'search_proper_nouns', 'args': {'query': 'alis in chain'}, 'id': 'fe02f7c4-1777-4a86-a0f1-846a0f6e5359', 'type': 'tool_call'}], usage_metadata={'input_tokens': 855, 'output_tokens': 48, 'total_tokens': 903})]}}
----
{'tools': {'messages': [ToolMessage(content='Alice In Chains\n\nDa Lama Ao Caos\n\nCorinne Bailey Rae\n\nKid Abelha\n\nAce Of Spades', name='search_proper_nouns', id='469ce1ba-2a16-4ae3-9334-e9d1bec15b